In [8]:
from tkinter import *
from random import random
import pandas as pd
import numpy as np


df = pd.read_csv("guidebook_score_export.csv", names = ['Judge_ID','Poster_ID','Presenter', 'Question_ID', 'Question', 'Response','Date'])

df=df[1:]
poster_list = np.unique(df["Poster_ID"])
poster_num = len(poster_list)
ROWS = int(np.sqrt(poster_num))+1
COLUMNS = int((poster_num-1)/ROWS)+1

color="purple"
default_color="white"
NUMJUDGES = 2
button_list = []
for i in range(ROWS*COLUMNS):
    button_list.append([default_color, 2])

buttons = []    
    
def main():
    window = Tk()
    Grid.rowconfigure(window, 0, weight=1)
    Grid.columnconfigure(window, 0, weight=1)
    frame=Frame(window)
    frame.grid(row=0, column=0, sticky=N+S+E+W)
    for x in range(ROWS):
        Grid.rowconfigure(frame, x, weight=1)
        for y in range(COLUMNS):
            if (x*ROWS+y) < poster_num:
                i = x*COLUMNS+y
                Grid.columnconfigure(frame, y, weight=1)
                b = Button(frame,text =str(i)+ ' ' + str(x)+str(y) + '- NEEDS ' + str(NUMJUDGES) + '\n'+poster_list[x*ROWS+y], bg=default_color, activebackground=default_color)
                b.grid(column=y, row=x)
                b["command"] = lambda b=b, x=x, y=y: click(b, x, y)
                b.grid(row=x, column=y, sticky=N+S+E+W)
                buttons.append(b)
    Grid.columnconfigure(frame, COLUMNS, weight=1)
    Grid.rowconfigure(frame, int(ROWS/2), weight=1)
    b = Button(frame, text = 'update', bg = default_color, activebackground = default_color)
    b["command"] = lambda : update()
    b.grid(column = COLUMNS, row = int(ROWS/2), sticky = N+S+E+W)
    return window

def click(button,x,y):
    i = x*COLUMNS+y
    print(x,y,i)
    if button_list[i][0] is color:
        button_list[i][0] = default_color
        button_list[i][1] -= 1
        button["bg"] = default_color
        button["activebackground"] = default_color
        button["text"] = str(x)+str(y)+ '- NEEDS ' + str(button_list[i][1]) + '\n'+poster_list[i]
    elif button_list[i][0] is default_color:
        button_list[i][0] = color
        button["bg"] = color
        button["activebackground"] = color
        button["text"] = str(x)+str(y) + '- JUDGING'
    if button_list[i][1] <= 0:
        button_list[i][0] = "grey"
        button["bg"] = "grey"
        button["activebackground"] = "grey"
        button["text"] = str(x)+str(y)+ '- DONE'
def update():
    random_list = np.asarray(np.random.choice(poster_num,size=10, replace=True),dtype=np.int)
    random_list= random_list.reshape(10)
    print(random_list)
    for i in random_list:
        button_list[i][1] -= 1
        button_column = int(i%COLUMNS)
        button_row = int((i-button_column)/COLUMNS)
        print(button_row,button_column,i)
        if button_list[i][1] <= 0:
            button_list[i][0] = "grey"
            button["bg"] = "grey"
            button["activebackground"] = "grey"
            button["text"] = str(i) + ' ' + str(x)+str(y)+ '- DONE'
        else:
            buttons[i]["text"]=str(i) + ' ' +str(button_row)+str(button_column)+ '- NEEDS ' + str(button_list[i][1]) + '\n'+poster_list[i]
    
w = main()
mainloop()
